# Experiments of the paper: "Efficient Detection of SQL Injection Attacks Utilising Cascade Natural Language Processing"

## Initializations

In [5]:
%load_ext autoreload
%autoreload 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from pathlib import Path
main_folder = Path('/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230415_sqli_colab')


In [7]:
%cd $main_folder
%ls -lah

/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230415_sqli_colab
total 133K
-rw------- 1 root root  683 Mar 31 15:15 classical_models.py
-rw------- 1 root root  587 Apr  9 12:15 config.ini
drwx------ 2 root root 4.0K Apr  9 12:12 datasets/
-rw------- 1 root root 7.6K Mar 31 15:15 ensemble_models.py
-rw------- 1 root root  13K Apr 10 16:42 experiments.py
-rw------- 1 root root  739 Mar 31 15:15 .gitignore
-rw------- 1 root root    3 Jan 19 13:28 LICENSE.md
-rw------- 1 root root  77K Apr 15 21:31 main.ipynb
drwx------ 2 root root 4.0K Apr 15 13:41 __pycache__/
-rw------- 1 root root 5.0K Apr  4 22:08 README.md
drwx------ 2 root root 4.0K Apr 15 20:54 results/
-rw------- 1 root root 1.3K Apr  4 22:08 run_classical_MLs.py
-rw------- 1 root root 5.8K Apr  4 22:08 templates.py
drwx------ 2 root root 4.0K Apr  9 12:12 trained_models/
drwx------ 2 root root 4.0K Apr 10 22:05 utils/


Warning: BERT libraries had problems after Colab system is upgraded. The solution I found is to downgrade numpy. After running the following code, restart the session and check the numpy version. If the libraries are working just fine in future, you can remove the downgrade fix.

In [5]:
!pip install -q -U numpy==1.22 --ignore-installed

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-model-optimization 0.7.4 requires numpy~=1.23, but you have numpy 1.22.0 which is incompatible.
librosa 0.10.0.post2 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.0 which is incompatible.


In [8]:
import numpy as np
import sklearn
print("numpy: \t\t", np.__version__)
print("sklearn: \t", sklearn.__version__)
!python --version

numpy: 		 1.22.0
sklearn: 	 1.2.2
Python 3.9.16


In [9]:
# A dependency of the preprocessing for BERT inputs
#!pip install  -q -U "tensorflow-text==2.8.*"
!pip install  -q -U "tensorflow-text"

In [10]:
#!pip install -q tf-models-official==2.7.0
!pip install -q tf-models-official

In [11]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.


In [12]:
from pathlib import Path
from datetime import datetime
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

import pandas as pd
import time


import time
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

from experiments import evaluate, save_results

print('Pandas v', pd.__version__)
#tf.get_logger().setLevel('ERROR')

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Pandas v 1.5.3


In [13]:
 !nvidia-smi

Sat Apr 15 21:31:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
import configparser
from templates import DataManager, logger
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [15]:
data_manager = DataManager(config)

In [16]:
#%%script echo skipping
results_dir = Path(config['results']['dir'])

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
file_name = f'results_{currentTime}.csv'
output_file = main_folder / results_dir / file_name
proposed_test_results_file = main_folder / results_dir / 'proposed_method_results.csv'
print(f"All results:{output_file.absolute()}")
print(f"Proposed method results:{proposed_test_results_file.absolute()}")



All results:/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230415_sqli_colab/results/results_230415_213200.csv
Proposed method results:/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230415_sqli_colab/results/proposed_method_results.csv


## Classical ML (Single NLP) tests

In [ ]:
!python run_classical_MLs.py -o $file_name

04-10-2023 16:55:03 l:26| Loaded dataset with 30609 rows.
Running the tests for seed: 42
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
04-10-2023 16:55:03 l:140| Training model: xgboost
04-10-2023 16:55:04 l:148| Ended training xgboost in: 1.1999408399999538s
04-10-2023 16:55:04 l:140| Training model: naive_bayes
04-10-2023 16:55:04 l:148| Ended training naive_bayes in: 0.0055923019999681856s
04-10-2023 16:55:04 l:140| Training model: svm
04-10-2023 16:55:50 l:148| Ended training svm in: 46.16930955499993s
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'wor

## LLM (BERT) tests

In [17]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
# Manual selection
bert_model_name_list = ['bert_en_uncased_L-12_H-768_A-12', 
                        'bert_en_cased_L-12_H-768_A-12',
                        'small_bert/bert_en_uncased_L-2_H-128_A-2',
                        'small_bert/bert_en_uncased_L-4_H-512_A-8',
                        'small_bert/bert_en_uncased_L-8_H-128_A-2',
                        'small_bert/bert_en_uncased_L-12_H-768_A-12',
                        'bert_multi_cased_L-12_H-768_A-12',
                       'albert_en_base',
                        'electra_base',
                        'electra_small']

In [18]:
def build_classifier_model(local_tfhub_handle_preprocess, local_tfhub_handle_encoder):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Payload')
  preprocessing_layer = hub.KerasLayer(local_tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(local_tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
data_manager.x_train.shape[0]

24487

In [ ]:
#main_dir = Path('/content/drive/MyDrive/Colab Notebooks/kasim/2023 Feb Bert SQLi/')
# 'SQLiV3_train.tsv', 'SQLiV3_test.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_testing.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
#train_file_name = 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
#train_file_name = 'SQLiV3_train.tsv'
#test_file_name = 'SQLiV3_test.tsv'
#train_file = Path(main_dir / train_file_name)
#test_file = Path(main_dir / test_file_name)

#print(f'Train file exists: {train_file.is_file()}')
#print(f'Test file exists: {test_file.is_file()}')


currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
#recording = Recorder(main_folder / results_dir / f'results_BERT_{currentTime}.csv')


AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
#seed = 42

x_train = data_manager.x_train
y_train = data_manager.y_train
x_test = data_manager.x_test
y_test = data_manager.y_test

# Create TensorFlow datasets for the training and validation sets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(8)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(8)

# Cache the training and validation datasets
train_dataset = train_dataset.cache()
val_dataset = val_dataset.cache()


# Prefetch the training and validation datasets
train_ds = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_ds = val_dataset.prefetch(buffer_size=AUTOTUNE)



#train_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = val_ds

class_names = ['payload','label']



In [ ]:
tmp = train_ds.take(1)
for i in next(iter(tmp)):
  print(i)

tf.Tensor(
[b'if  (  6554  =  1534  )   select 6554 else drop function buqb--'
 b"admin'  )   or   (  '1'  =  '1'--"
 b' select * from users where id  =  1 or "&  )  " or 1  =  1 -- 1'
 b'callejon sur, 93 13-c' b"-6449'   )    )    or 2590  =  2848"
 b"1' in boolean mode  ) "
 b"1' and 5556  =    (  select count  (  *  )   from all_users t1,all_users t2,all_users t3,all_users t4,all_users t5  )  "
 b'SELECT MIN ( wood )  AS union FROM disease'], shape=(8,), dtype=string)
tf.Tensor([1 1 1 0 1 1 1 0], shape=(8,), dtype=int64)


In [ ]:
for bert_model_name in bert_model_name_list:
  #bert_model_name = bert_model_name_list[0]
  print('****************************************************')
  #recording.set_current_method(f"{bert_model_name}" )
  tfhub_handle_encoder = map_name_to_handle[bert_model_name]
  tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

  print(f'BERT model selected           : {tfhub_handle_encoder}')
  print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


  bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
  bert_model = hub.KerasLayer(tfhub_handle_encoder)
  classifier_model = build_classifier_model(tfhub_handle_preprocess, tfhub_handle_encoder)

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  metrics = tf.metrics.BinaryAccuracy()

  epochs = 10 #modify this
  #steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
  steps_per_epoch = data_manager.x_train.shape[0]
  num_train_steps = (steps_per_epoch * epochs) #//200 
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')

  classifier_model.compile(optimizer=optimizer,
                          loss=loss,
                          metrics=metrics)


  print(f'Training model with {tfhub_handle_encoder}')
  start_time = time.time()
  history = classifier_model.fit(x=train_ds, #train_ds.take(2)
                                validation_data=test_ds, # validation_data=test_ds.take(2),
                                epochs=epochs)
  stop_time = time.time()
  training_time = (stop_time - start_time)*1000 / data_manager.notes['train_size']

  loss, accuracy = classifier_model.evaluate(test_ds)

  print(f'Loss: {loss}')
  print(f'Accuracy: {accuracy}')

  start_time = time.time()
  y_pred = classifier_model.predict(test_ds)
  stop_time = time.time()
  testing_time = (stop_time - start_time)*1000 / data_manager.notes['test_size']
  y_pred_np = tf.cast(tf.sigmoid(y_pred) > 0.5, tf.int32).numpy()
  X, y_true = zip(*test_ds.unbatch())
  y_true_np = [y.numpy() for y in y_true]
  #precision, recall, f1, support = precision_recall_fscore_support(y_true_np, y_pred_np, average='binary')
  #tn, fp, fn, tp = confusion_matrix(y_true_np, y_pred_np).ravel()

  notes = {'feature_method': bert_model_name,'model': bert_model_name, 
           'seed': data_manager.notes['seed'], 
           'split_ratio': data_manager.notes['split_ratio'],
           'train_size': data_manager.notes['train_size'],
           'test_size': data_manager.notes['test_size'],
           'extraction_time':0, 'feature_size':0,
           'train_time': training_time,
           'pred_time': testing_time, 'dataset': config['dataset']['file']}
  result = evaluate(y_true_np, y_pred_np, notes)
  print(result)
  save_results([result], output_file)


****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061 [==============================] - 189s 56ms/step - loss: 0.1124 - binary_accuracy: 0.9591 - val_loss: 0.0236 - val_binary_accuracy: 0.9967
Epoch 2/10
3061/3061 [==============================] - 172s 56ms/step - loss: 0.0151 - binary_accuracy: 0.9972 - val_loss: 0.0171 - val_binary_accuracy: 0.9980
Epoch 3/10
3061/3061 [==============================] - 171s 56ms/step - loss: 0.0092 - binary_accuracy: 0.9984 - val_loss: 0.0179 - val_binary_accuracy: 0.9977
Epoch 4/10
3061/3061 [==============================] - 171s 56ms/step - loss: 0.0064 - binary_accuracy: 0.9989 - val_loss: 0.0166 - val_binary_accuracy: 0.9982
Epoch 5/10
3061/3061 [=============

## The Proposed Cascade NLP (two-stage) SQLi Detection Tests

In [19]:
# Data manager _ split with seed
from sklearn.model_selection import train_test_split

def split_dataset(seed=42):
  split_ratio = float(data_manager.config['data_manager']['split_ratio'])

  data_manager.train, data_manager.test = train_test_split(
    data_manager.dataset, test_size=split_ratio, random_state=seed)

  data_manager.x_train = data_manager.train['payload'].values
  data_manager.x_test = data_manager.test['payload'].values
  data_manager.y_train = data_manager.train['label'].values
  data_manager.y_test = data_manager.test['label'].values

  data_manager.notes = {
    'seed': seed,
    'split_ratio': split_ratio,
    'train_size': len(data_manager.train),
    'test_size': len(data_manager.test),
  }

#split_dataset(seed=666)
#print(data_manager.notes['seed'])

### First Stage - Classical ML

In [20]:
from templates import FeatureExtractor
from classical_models import Classical_Model
from pprint import pprint
from experiments import evaluate

threshold = 0.05 #prediction
scale_pos_weight = 5000.0

feature_method = 'tf-idf_ngram'
model_name = 'xgboost'

In [24]:
# Train and evaluate the first stage
def first_stage():
  feature_extractor = FeatureExtractor(feature_method)
  start_time = time.time()
  feature_extractor.extract_features(
      data_manager.x_train, data_manager.x_test)
  stop_time = time.time()
  extraction_time = ((stop_time - start_time)*1000 
                    / (len(data_manager.x_train) + len(data_manager.x_test)) )

  model = Classical_Model(model_name, scale_pos_weight=scale_pos_weight)
  model.feature_method = feature_extractor.method
  start_time = time.time()
  model.fit(
      feature_extractor.features['train'], 
      data_manager.y_train) #scale_pos_weight=5.0
  stop_time = time.time()
  training_time = (stop_time - start_time)*1000 / feature_extractor.features['train'].shape[0]


  start_time = time.time()
  first_stage_y_pred = model.predict(feature_extractor.features['test'], threshold=threshold)
  stop_time = time.time()

  testing_time = (stop_time - start_time)*1000 / feature_extractor.features['test'].shape[0]

  notes = {'feature_method': feature_extractor.method,'model': model_name, 
            'seed': data_manager.notes['seed'], 
            'split_ratio': data_manager.notes['split_ratio'],
            'train_size': data_manager.notes['train_size'],
            'test_size': data_manager.notes['test_size'],
            'extraction_time':extraction_time, 'feature_size': feature_extractor.features['train'].shape[1],
            'train_time': training_time,
            'pred_time': testing_time, 'dataset': config['dataset']['file'],
          'scale_pos_weight':scale_pos_weight,
          'threshold': threshold
          }
  # Save results to csv file

  result = evaluate(data_manager.y_test, first_stage_y_pred, notes=notes)
  pprint(result)
  save_results([result], proposed_test_results_file)

  # Extract the positive predicitions for the second stage
  first_stage_positive_preds = data_manager.x_test[(first_stage_y_pred == 1)]
  first_stage_positive_preds_true_labels = data_manager.y_test[(first_stage_y_pred == 1)]
  fs_pos_len = len(first_stage_positive_preds)
  nof_test_samples = len(data_manager.x_test)
  fs_rat = fs_pos_len/nof_test_samples
  print(f"Positive predicitions in the first stage: {fs_pos_len} out of {nof_test_samples}. Ratio:{fs_rat}")

  return first_stage_positive_preds, first_stage_positive_preds_true_labels

### Second Stage - BERT

In [27]:
def second_stage(first_stage_positive_preds, first_stage_positive_preds_true_labels):
  # Prepare the training Dataset 
  currentDateAndTime = datetime.now()
  currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
  #recording = Recorder(main_folder / results_dir / f'results_BERT_{currentTime}.csv')


  AUTOTUNE = tf.data.AUTOTUNE
  batch_size = 8
  #seed = 42

  x_train = data_manager.x_train
  y_train = data_manager.y_train
  x_test = data_manager.x_test
  y_test = data_manager.y_test

  # Create TensorFlow datasets for the training and validation sets
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
  val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

  # Cache the training and validation datasets
  train_dataset = train_dataset.cache()
  val_dataset = val_dataset.cache()


  # Prefetch the training and validation datasets
  train_ds = train_dataset.prefetch(buffer_size=AUTOTUNE)
  test_ds = val_dataset.prefetch(buffer_size=AUTOTUNE)

  class_names = ['payload','label']

  # First, train the BERT model.

  print('****************************************************')
  bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'
  tfhub_handle_encoder = map_name_to_handle[bert_model_name]
  tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

  print(f'BERT model selected           : {tfhub_handle_encoder}')
  print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


  bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
  bert_model = hub.KerasLayer(tfhub_handle_encoder)
  second_stage_classifier_model = build_classifier_model(tfhub_handle_preprocess, tfhub_handle_encoder)

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  metrics = tf.metrics.BinaryAccuracy()

  epochs = 10 #DEBUG MODIFY ME
  #steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
  steps_per_epoch = data_manager.x_train.shape[0]
  num_train_steps = (steps_per_epoch * epochs) #//200 
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')

  second_stage_classifier_model.compile(optimizer=optimizer,
                          loss=loss,
                          metrics=metrics)


  print(f'Training model with {tfhub_handle_encoder}')
  start_time = time.time()
  history = second_stage_classifier_model.fit(x=train_ds, # x=train_ds.take(2), 
                                validation_data=test_ds, # validation_data=test_ds.take(2), 
                                epochs=epochs)
  stop_time = time.time()
  training_time = (stop_time - start_time)*1000 / data_manager.notes['train_size']

  # Prepare first stage outputs for the second stage
  print(len(first_stage_positive_preds)) #np array
  print(len(first_stage_positive_preds_true_labels)) # np array

  # Create TensorFlow datasets for the prediction set
  pred_dataset = tf.data.Dataset.from_tensor_slices(
      (first_stage_positive_preds, first_stage_positive_preds_true_labels)).batch(batch_size)

  # Cache the pred dataset
  pred_dataset = pred_dataset.cache()

  # Prefetch the pred datasets
  first_stage_pos_preds = pred_dataset.prefetch(buffer_size=AUTOTUNE)

  # Predict only positive outputs of the first stage
  start_time = time.time()
  y_pred = second_stage_classifier_model.predict(first_stage_pos_preds)#.take(2))
  stop_time = time.time()
  testing_time = (stop_time - start_time)*1000 / len(first_stage_positive_preds) 
  y_pred_np = tf.cast(tf.sigmoid(y_pred) > 0.5, tf.int32).numpy()
  X, y_true = zip(*first_stage_pos_preds.unbatch())# take(2).
  y_true_np = [y.numpy() for y in y_true]
  #precision, recall, f1, support = precision_recall_fscore_support(y_true_np, y_pred_np, average='binary')
  #tn, fp, fn, tp = confusion_matrix(y_true_np, y_pred_np).ravel()

  notes = {'feature_method': bert_model_name,'model': bert_model_name, 
            'seed': data_manager.notes['seed'], 
            'split_ratio': data_manager.notes['split_ratio'],
            'train_size': data_manager.notes['train_size'],
            'test_size': data_manager.notes['test_size'],
            'extraction_time':0, 'feature_size':0,
            'train_time': training_time,
            'pred_time': testing_time, 'dataset': config['dataset']['file']}
  result = evaluate(y_true_np, y_pred_np, notes)
  pprint(result)
  save_results([result], proposed_test_results_file)


In [ ]:
# Run first and second stages with different seeds:
seeds = [23, 6, 34, 1984, 1994, 77]
for seed in seeds:
  print('######################################')
  print(f"seed:{seed}")
  split_dataset(seed=seed)
  print(data_manager.notes['seed'])
  first_stage_y_pred, first_stage_positive_preds_true_labels = first_stage()
  second_stage(first_stage_y_pred, first_stage_positive_preds_true_labels)

######################################
seed:23
23


/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


{'accuracy': 0.9862789937928781,
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0.08238786935860798,
 'f1_score': 0.9819742489270387,
 'feature_method': 'tf-idf_ngram',
 'feature_size': 29074,
 'fn': 3,
 'fp': 81,
 'model': 'xgboost',
 'precision': 0.9658083579569439,
 'pred_time': 0.002254227803681108,
 'recall': 0.9986905281536447,
 'scale_pos_weight': 5000.0,
 'seed': 23,
 'split_ratio': 0.2,
 'test_size': 6122,
 'threshold': 0.05,
 'tn': 3750,
 'tp': 2288,
 'train_size': 24487,
 'train_time': 0.577415710968293}
Positive predicitions in the first stage: 2369 out of 6122. Ratio:0.38696504410323423
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061 [==============================] - 188s 56ms/step - 

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


{'accuracy': 0.9851355766089513,
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0.0834205024497652,
 'f1_score': 0.9805264284185747,
 'feature_method': 'tf-idf_ngram',
 'feature_size': 29196,
 'fn': 6,
 'fp': 85,
 'model': 'xgboost',
 'precision': 0.9642255892255892,
 'pred_time': 0.0019423987342812041,
 'recall': 0.9973878972572922,
 'scale_pos_weight': 5000.0,
 'seed': 6,
 'split_ratio': 0.2,
 'test_size': 6122,
 'threshold': 0.05,
 'tn': 3740,
 'tp': 2291,
 'train_size': 24487,
 'train_time': 0.48259850076449606}
Appending to the existing .csv file.
Positive predicitions in the first stage: 2376 out of 6122. Ratio:0.38810846128716103
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061 [=============

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


{'accuracy': 0.9857889578569095,
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0.0829493046476753,
 'f1_score': 0.9810992830762545,
 'feature_method': 'tf-idf_ngram',
 'feature_size': 27216,
 'fn': 4,
 'fp': 83,
 'model': 'xgboost',
 'precision': 0.964545066211021,
 'pred_time': 0.004984124428849717,
 'recall': 0.9982316534040672,
 'scale_pos_weight': 5000.0,
 'seed': 34,
 'split_ratio': 0.2,
 'test_size': 6122,
 'threshold': 0.05,
 'tn': 3777,
 'tp': 2258,
 'train_size': 24487,
 'train_time': 0.5836119651599703}
Appending to the existing .csv file.
Positive predicitions in the first stage: 2341 out of 6122. Ratio:0.38239137536752693
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061 [===============

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


{'accuracy': 0.9872590656648155,
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0.08288315905083352,
 'f1_score': 0.9830065359477125,
 'feature_method': 'tf-idf_ngram',
 'feature_size': 29147,
 'fn': 4,
 'fp': 74,
 'model': 'xgboost',
 'precision': 0.9682403433476395,
 'pred_time': 0.0021736125703030883,
 'recall': 0.9982300884955753,
 'scale_pos_weight': 5000.0,
 'seed': 1984,
 'split_ratio': 0.2,
 'test_size': 6122,
 'threshold': 0.05,
 'tn': 3788,
 'tp': 2256,
 'train_size': 24487,
 'train_time': 0.5747240745515457}
Appending to the existing .csv file.
Positive predicitions in the first stage: 2330 out of 6122. Ratio:0.38059457693564197
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061 [==========